In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# installing offline dependencies
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn tqdm

In [ ]:
!pip install accelerate bitsandbytes pydantic

In [ ]:
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
from __future__ import annotations

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")

In [ ]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [ ]:
SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16

WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH)

model_dir = "/kaggle/input/llm-se-debertav3-large"

In [ ]:
# sentence_model = SentenceTransformer(SIM_MODEL, device='cuda')
# sentence_model.max_seq_length = MAX_LENGTH
# sentence_model = sentence_model.half() # half precision

In [ ]:
# sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

## below this need to do in every API calls

In [ ]:
# change the trn.prompt.values to a single prompt (question and every choice is space separated)
# prompt_embeddings = sentence_model.encode(
#     trn.prompt.values,
#     batch_size=BATCH_SIZE,
#     device=DEVICE,
#     show_progress_bar=True,
#     convert_to_tensor=True,
#     normalize_embeddings=True,
# )
# prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
# _ = gc.collect()

In [ ]:
# ## Get the top 3 pages that are likely to contain the topic of interest
# search_score, search_index = sentence_index.search(prompt_embeddings, 3)

Getting relevent titles

In [ ]:
# # it's better if we keep this in memory
# df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
#                      columns=['id', 'file'])

In [ ]:
# ## Get the article and associated file location using the index
# wikipedia_file_data = []

# for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
#     scr_idx = idx
#     _df = df.loc[scr_idx].copy()
#     _df['prompt_id'] = i
#     wikipedia_file_data.append(_df)
# wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
# wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

# ## Save memory - delete df since it is no longer necessary
# # del df
# _ = gc.collect()
# # libc.malloc_trim(0)

In [ ]:
# ## Get the full text data
# wiki_text_data = []

# for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
#     _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
#     _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

#     _df_temp = _df[_df['id'].isin(_id)].copy()
#     del _df
#     _ = gc.collect()
#     libc.malloc_trim(0)
#     wiki_text_data.append(_df_temp)
# wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
# _ = gc.collect()

In [ ]:
# ## Parse documents into sentences
# processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values)

In [ ]:
# ## Get embeddings of the wiki text data
# wiki_data_embeddings = model.encode(processed_wiki_text_data.text,
#                                     batch_size=BATCH_SIZE,
#                                     device=DEVICE,
#                                     show_progress_bar=True,
#                                     convert_to_tensor=True,
#                                     normalize_embeddings=True)#.half()
# wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()

In [ ]:
# _ = gc.collect()

In [ ]:
# ## change this to create only a single prompt

# ## Combine all answers
# trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)


# ## Search using the prompt and answers to guide the search
# trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

In [ ]:
# question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
# question_embeddings = question_embeddings.detach().cpu().numpy()

In [ ]:
# ## change this as well for a single prompt

# ## Parameter to determine how many relevant sentences to include
# NUM_SENTENCES_INCLUDE = 5

# ## List containing just Context
# contexts = []

# for r in tqdm(trn.itertuples(), total=len(trn)):

#     prompt_id = r.Index

#     prompt_indices = processed_wiki_text_data[processed_wiki_text_data['document_id'].isin(wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values)].index.values

#     if prompt_indices.shape[0] > 0:
#         prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
#         prompt_index.add(wiki_data_embeddings[prompt_indices])

#         context = ""
        
#         ## Get the top matches
#         ss, ii = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
#         for _s, _i in zip(ss[prompt_id], ii[prompt_id]):
#             context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + " "
        
#     contexts.append(context)

## Model

In [ ]:
# model_dir = "/kaggle/input/llm-se-debertav3-large"

In [ ]:
# from dataclasses import dataclass
# from typing import Optional, Union

# import torch
# import numpy as np
# import pandas as pd
# from datasets import Dataset
# from transformers import AutoTokenizer
# from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
# from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

In [ ]:
# test_df = pd.read_csv("test_context.csv")
# test_df.index = list(range(len(test_df)))
# test_df.id = list(range(len(test_df)))
# test_df["prompt"] = test_df["context"] + " #### " +  test_df["prompt"]

### didn't use the following yet

In [ ]:
# test_df['answer'] = 'A'
# test_ds = Dataset.from_pandas(test_df)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_dir)
# model = AutoModelForMultipleChoice.from_pretrained(model_dir)
# model.eval()

In [ ]:
# import numpy as np
# def predictions_to_map_output(predictions):
#     sorted_answer_indices = np.argsort(-predictions)
#     top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
#     top_answers = np.vectorize(index_to_option.get)(top_answer_indices)
#     return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

In [ ]:
# # We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
# options = 'ABCDE'
# indices = list(range(5))

# option_to_index = {option: index for option, index in zip(options, indices)}
# index_to_option = {index: option for option, index in zip(options, indices)}

# def preprocess(example):
#     # The AutoModelForMultipleChoice class expects a set of question/answer pairs
#     # so we'll copy our question 5 times before tokenizing
#     first_sentence = [example['prompt']] * 5
#     second_sentence = []
#     for option in options:
#         second_sentence.append(example[option])
#     # Our tokenizer will turn our text into token IDs BERT can understand
#     tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
#     tokenized_example['label'] = option_to_index[example['answer']]
#     return tokenized_example

In [ ]:
# @dataclass
# class DataCollatorForMultipleChoice:
#     tokenizer: PreTrainedTokenizerBase
#     padding: Union[bool, str, PaddingStrategy] = True
#     max_length: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
    
#     def __call__(self, features):
#         label_name = "label" if 'label' in features[0].keys() else 'labels'
#         labels = [feature.pop(label_name) for feature in features]
#         batch_size = len(features)
#         num_choices = len(features[0]['input_ids'])
#         flattened_features = [
#             [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
#         ]
#         flattened_features = sum(flattened_features, [])
        
#         batch = self.tokenizer.pad(
#             flattened_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors='pt',
#         )
#         batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
#         batch['labels'] = torch.tensor(labels, dtype=torch.int64)
#         return batch

In [ ]:
# model = AutoModelForMultipleChoice.from_pretrained(f'/kaggle/input/science-exam-trained-model-weights/run_2').cuda()

In [ ]:
# trainer = Trainer(
#     model=model,
#     tokenizer=tokenizer,
#     data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer)
# )

In [ ]:
# test_predictions = trainer.predict(tokenized_test_ds)

In [ ]:
# submission_df = pd.DataFrame({"id": np.arange(len(test_df))})
# submission_df['prediction'] = predictions_to_map_output(test_predictions.predictions)

# submission_df.head()

# Fast API

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel, validator
from fastapi.middleware.cors import CORSMiddleware

import os
import requests
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

from dataclasses import dataclass
from typing import Optional, Union
from tqdm import tqdm

import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy


class MCQ(BaseModel):
    question: str
    choices: list[str]

    @validator("choices")
    def validate_choices_length(cls, choices):
        min_length = 1  # Minimum allowed length
        max_length = 5  # Maximum allowed length

        if len(choices) < min_length:
            raise ValueError(f"Choices list must have at least {min_length} item(s)")
        if len(choices) > max_length:
            raise ValueError(f"Choices list can have at most {max_length} items")

        return choices

    @validator("question")
    def validate_question_length(cls, question):
        min_length = 1  # Minimum allowed length
        max_length = 512  # Maximum allowed length

        if len(question) < min_length:
            raise ValueError(f"Question must have at least {min_length} character(s)")
        if len(question) > max_length:
            raise ValueError(f"Question can have at most {max_length} characters")

        return question


app = FastAPI(
    title="Scique API",
    description="An API to answer any MCQ from wikipedia without context and other questions with context.",
    version="0.1.0",
    docs_url="/docs",
    redoc_url="/redoc",
)

origins = ["http://localhost:8000", "localhost:8000"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# eluwa_model: AutoModelForCausalLM | None = None
# tokenizer: AutoTokenizer | None = None

# sentence_model: SentenceTransformer | None = None
# sentence_index: faiss.Index | None = None

wiki_parquet: pd.DataFrame | None = None
sentence_index_address: str| None = None


# load model artifacts on startup of the application to reduce latency
@app.on_event("startup")
async def startup_event():
    global sentence_index_address, wiki_parquet

    wiki_parquet = pd.read_parquet(
        "/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
        columns=["id", "file"],
    )
    
    sentence_index_address = input("Enter address to wiki-faiss index search url: ")


@app.get("/", tags=["root"])
async def root() -> str:
    return "Welcome to Scique API!"


@app.put("/askLLM/", tags=["askLLM"])
async def ask_mcq(mcq: MCQ | None):
    global wiki_parquet, sentence_index_address

    response = requests.put(sentence_index_address + "/search_wiki/", json=mcq.dict())
    
    if response.status_code == 200:
        print("PUT request was successful.")
    else:
        print(f"PUT request failed with status code {response.status_code}")
        
    data = response.json()
    
    search_score, search_index = data[0][0], data[1][0]
    
    search_score = [[float(x) for x in search_score]]
    search_index = [[int(x) for x in search_index]]

    ## Get the article and associated file location using the index
    wikipedia_file_data = []

    for i, (scr, idx) in tqdm(
        enumerate(zip(search_score, search_index)), total=len(search_score)
    ):
        scr_idx = idx
        _df = wiki_parquet.loc[scr_idx].copy()
        _df["prompt_id"] = i
        wikipedia_file_data.append(_df)
    wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
    wikipedia_file_data = (
        wikipedia_file_data[["id", "prompt_id", "file"]]
        .drop_duplicates()
        .sort_values(["file", "id"])
        .reset_index(drop=True)
    )

    _ = gc.collect()
    # libc.malloc_trim(0)

    ## Get the full text data
    wiki_text_data = []

    for file in tqdm(
        wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())
    ):
        _id = [
            str(i)
            for i in wikipedia_file_data[wikipedia_file_data["file"] == file][
                "id"
            ].tolist()
        ]
        _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=["id", "text"])

        _df_temp = _df[_df["id"].isin(_id)].copy()
        del _df
        _ = gc.collect()
        libc.malloc_trim(0)
        wiki_text_data.append(_df_temp)
    wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)

    _ = gc.collect()

    ## Parse documents into sentences
    processed_wiki_text_data = process_documents(
        wiki_text_data.text.values, wiki_text_data.id.values
    )
    
    return processed_wiki_text_data

#     ## Get embeddings of the wiki text data
#     wiki_data_embeddings = model.encode(
#         processed_wiki_text_data.text,
#         batch_size=BATCH_SIZE,
#         device=DEVICE,
#         show_progress_bar=True,
#         convert_to_tensor=True,
#         normalize_embeddings=True,
#     )  # .half()
#     wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()

#     _ = gc.collect()

#     question_embeddings = model.encode(
#         [question],
#         batch_size=BATCH_SIZE,
#         device=DEVICE,
#         show_progress_bar=True,
#         convert_to_tensor=True,
#         normalize_embeddings=True,
#     )
#     question_embeddings = question_embeddings.detach().cpu().numpy()

#     ## Parameter to determine how many relevant sentences to include
#     NUM_SENTENCES_INCLUDE = 5

#     ## List containing just Context
#     # contexts = []

#     prompt_id = 0

#     prompt_indices = processed_wiki_text_data[
#         processed_wiki_text_data["document_id"].isin(
#             wikipedia_file_data[wikipedia_file_data["prompt_id"] == prompt_id][
#                 "id"
#             ].values
#         )
#     ].index.values

#     if prompt_indices.shape[0] > 0:
#         prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
#         prompt_index.add(wiki_data_embeddings[prompt_indices])

#         context = ""

#         ## Get the top matches
#         ss, ii = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
#         for _s, _i in zip(ss[prompt_id], ii[prompt_id]):
#             context += (
#                 processed_wiki_text_data.loc[prompt_indices]["text"].iloc[_i] + " "
#             )

#     # contexts.append(context)
#     prompt = context + " #### " + mcq.question

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

nest_asyncio.apply()


# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)

# finally run the app
uvicorn.run(app, port=port)
